<h1 style="background-color: #f0f0f0; color: #000000; padding: 30px; border-radius: 15px; text-align: center; font-family: 'JetBrains Mono', sans-serif; font-size: 2rem;">
    Video Ad Engagement Vertex AI AutoML
</h1>

<div style="background-color: #f0f0f0; box-shadow: 10px 5px; padding: 20px; border-radius: 15px; margin: auto; width: 100vw">
    <p style="font-family: 'JetBrains Mono', sans-serif; font-size: 1.2rem; color: #000000">
        <b>About This Notebook:</b>
    </p>
    <p style="font-family: 'JetBrains Mono', sans-serif; font-size: 1rem; color: #333333;">
        This notebook focuses on using Google Cloud Platform's (GCP) Vertex AI to train a model using the AutoML feature. Then, after the model is created, it can later be exported to non-cloud machines for predictions. The data source for this notebook is <a href="https://www.kaggle.com/datasets/karnikakapoor/video-ads-engagement-dataset?select=ad_df.csv" target="_blank">Video Ad Engagement Dataset</a> on Kaggle.
    </p>
</div>

<div style="background-color: #f0f0f0; box-shadow: 10px 5px; padding: 20px; border-radius: 15px; margin: auto; width: 100vw">
    <p style="font-family: 'JetBrains Mono', sans-serif; font-size: 1.2rem; color: #000000">
        <b>Environment Setup:</b>
    </p>
    <p style="font-family: 'JetBrains Mono', sans-serif; font-size: 1rem; color: #333333;">
        This notebook is intended to be run within a notebook instance within GCP. As such, the <code>Managing Dependencies</code> cell can only be executed within that environment, as it has a multitude of dependencies that are already handled by GCP.
    </p>
    <p style="font-family: 'JetBrains Mono', sans-serif; font-size: 1rem; color: #333333;">
        Additionally, the constants in the <code>Setting Up Environment</code> can be adjusted where necessary to match values in your project.
    </p>
</div>

In [ ]:
### Managing Dependencies
## Installing/Upgrading Libraries
# GCP SDK
! pip3 install --upgrade google-cloud-aiplatform

In [ ]:
### Imports
## External Packages
from google.cloud import aiplatform

In [1]:
### Setting Up Environment
## Variables
# Project
PROJECT_ID = "video-ad-automl"

# Data
DATASET_NAME = "video_ad_engagement"
TABLE_NAME = "ad_engagement"
BIGQUERY_URI = f"bq://{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}"

# Model
MODEL_VERSION = 1
TRAINING_TIME = 1000
TRANSFORMATIONS = [
    {"auto": {"column_name": "auction_id"}},
    {"auto": {"column_name": "timestamp"}},
    {"auto": {"column_name": "creative_duration"}},
    {"auto": {"column_name": "creative_id"}},
    {"auto": {"column_name": "campaign_id"}},
    {"auto": {"column_name": "advertiser_id"}},
    {"auto": {"column_name": "placement_id"}},
    {"auto": {"column_name": "placement_language"}},
    {"auto": {"column_name": "website_id"}},
    {"auto": {"column_name": "referer_deep_three"}},
    {"auto": {"column_name": "ua_country"}},
    {"auto": {"column_name": "ua_os"}},
    {"auto": {"column_name": "ua_browser"}},
    {"auto": {"column_name": "ua_browser_version"}},
    {"auto": {"column_name": "ua_device"}},
    {"auto": {"column_name": "user_average_seconds_played"}}
]
TARGET = "seconds_played"

<div style="background-color: #f0f0f0; box-shadow: 10px 5px; padding: 20px; border-radius: 15px; margin: auto; width: 100vw">
    <p style="font-family: 'JetBrains Mono', sans-serif; font-size: 1.2rem; color: #000000">
        <b>Data Preparation:</b>
    </p>
    <p style="font-family: 'JetBrains Mono', sans-serif; font-size: 1rem; color: #333333;">
        This section creates a dataset accessible to the Vertex AI API, from data stored in BigQuery.
    </p>
</div>

In [ ]:
### Data Preparations
## Creating Managed Dataset
# Ad Engagement Data
dataset = aiplatform.TabularDataset.create(
    display_name = TABLE_NAME,
    bq_source = [BIGQUERY_URI]
)

<div style="background-color: #f0f0f0; box-shadow: 10px 5px; padding: 20px; border-radius: 15px; margin: auto; width: 100vw">
    <p style="font-family: 'JetBrains Mono', sans-serif; font-size: 1.2rem; color: #000000">
        <b>Model Training:</b>
    </p>
    <p style="font-family: 'JetBrains Mono', sans-serif; font-size: 1rem; color: #333333;">
        This section creates a training job that will produce a model.
    </p>
</div>

In [ ]:
### Machine Learning
## Training
# Defining Training Job
job = aiplatform.AutoMLTabularTrainingJob(
    display_name = f"video_ad_engagement_training_v{MODEL_VERSION}",
    optimization_prediction_type = "regression",
    column_transformations = TRANSFORMATIONS,
    optimization_objective = "minimize-mae"
)

# Running Training Job
job.run(
    dataset = dataset,
    target_column = TARGET,
    budget_milli_node_hours = TRAINING_TIME,
    model_display_name = f"video_ad_engagement_model_v{MODEL_VERSION}",
    disable_early_stopping = False
)